# Create Azure Cognitive Service for Face 

Before you start this lab, please take 6 photos of yourself and upload thoes to Azure Blob storage account.

You'll use your photo to train the cognitive service.

> Note that each photo must have single person

# Create group

To train face of a person, you need to create group that a person wil be assigned

To make REST API call, you need Face API service Key. 

You can get the key from the Face API at Azure Portal.


In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
import uuid
import json

headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': 'FaceAPIServiceKey',
}

params = urllib.parse.urlencode({
})

group_id = str(uuid.uuid1())

_body = '''
{
    "name": "VIP-Membership",
    "userData": "User-provided data attached to the large person group.",
    "recognitionModel": "recognition_01"
}
'''

try:
    conn = http.client.HTTPSConnection('westus2.api.cognitive.microsoft.com')
    conn.request("PUT", "/face/v1.0/largepersongroups/"+group_id, _body, headers)
    response = conn.getresponse()
    data = response.read()
    print(response.getcode())
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

# List the group 

In [ ]:

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': 'FaceAPIServiceKey',
}

params = urllib.parse.urlencode({
    # Request parameters
    'start': 'V',
    'top': '5',
    'returnRecognitionModel': 'true',
})

try:
    conn = http.client.HTTPSConnection('westus2.api.cognitive.microsoft.com')
    conn.request("GET", "/face/v1.0/largepersongroups", "", headers)
    response = conn.getresponse()
    data = response.read()
    for rec in json.loads(data):
        print(rec)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

# Create user before upload photo

In [ ]:
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': 'FaceAPIServiceKey',
}

params = urllib.parse.urlencode({
})

_body='''
{
    "name": "name",
    "userData": "User-provided data attached to the person."
}
'''

user_id = ""

try:
    conn = http.client.HTTPSConnection('westus2.api.cognitive.microsoft.com')
    conn.request("POST", "/face/v1.0/largepersongroups/"+group_id+"/persons?%s" % params, _body, headers)
    response = conn.getresponse()
    user_id = json.loads(response.read())["personId"]
    print(user_id)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

# Upload photo

In [ ]:
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': 'FaceAPIServiceKey',
}

params = urllib.parse.urlencode({
    # Request parameters
#     'userData': '{string}',
#     'targetFace': '{string}',
})

images = ['https://azlab0aml9378917344.blob.core.windows.net/pics/20190427_225442.jpg',
          'https://azlab0aml9378917344.blob.core.windows.net/pics/20190427_225450.jpg',
          'https://azlab0aml9378917344.blob.core.windows.net/pics/20190427_225454.jpg',
          'https://azlab0aml9378917344.blob.core.windows.net/pics/20190427_225506.jpg',
          'https://azlab0aml9378917344.blob.core.windows.net/pics/20190427_225507.jpg',
         ]


for img in images:
    _body = "{\"url\": \""+img+"\"}"

    try:
        conn = http.client.HTTPSConnection('westus2.api.cognitive.microsoft.com')
        conn.request("POST", "/face/v1.0/largepersongroups/"+group_id+"/persons/"+user_id+"/persistedfaces", _body, headers)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))   
    

# Start training

In [ ]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': 'FaceAPIServiceKey',
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('westus2.api.cognitive.microsoft.com')
    conn.request("POST", "/face/v1.0/largepersongroups/"+group_id+"/train?%s" % params, "", headers)
    response = conn.getresponse()
    data = response.read()
    print(response.getcode())
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

# Check training status

In [ ]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': 'FaceAPIServiceKey',
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('westus2.api.cognitive.microsoft.com')
    conn.request("GET", "/face/v1.0/largepersongroups/"+group_id+"/training?%s" % params, "", headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

# Get new face id to identify from a group

In [ ]:
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': 'FaceAPIServiceKey',
}

params = urllib.parse.urlencode({
    # Request parameters
    'returnFaceId': 'true',
#     'returnFaceLandmarks': 'false',
#     'returnFaceAttributes': '{string}',
#     'recognitionModel': 'recognition_02',
#     'returnRecognitionModel': 'false',
})

_body = '''
{
    "url": "https://azlab0aml9378917344.blob.core.windows.net/pics/20190427_225439.jpg"
}
'''

_faceId = ""

try:
    conn = http.client.HTTPSConnection('westus2.api.cognitive.microsoft.com')
    conn.request("POST", "/face/v1.0/detect?%s" % params, _body, headers)
    response = conn.getresponse()
    data = response.read()
    _faceId = json.loads(data)[0]['faceId']
    print(_faceId)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

# Run prediction

In [ ]:
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': 'FaceAPIServiceKey',
}

params = urllib.parse.urlencode({
})

_body="{ \"largePersonGroupId\": \""+group_id+"\", \"faceIds\": [\""+_faceId+"\"],  \"maxNumOfCandidatesReturned\": 2, \"confidenceThreshold\": 0.5 }"


try:
    conn = http.client.HTTPSConnection('westus2.api.cognitive.microsoft.com')
    conn.request("POST", "/face/v1.0/identify?%s" % params, _body, headers)
    response = conn.getresponse()
    data = response.read()
    print(json.loads(data)[0])
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))